In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [2]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [3]:
files = os.listdir("dogs-vs-cats/train/")

In [4]:
categories = []
for file_name in files:
    file_category = file_name.split('.')[0]
    if file_category == "cat":
        categories.append("cat")
    else:
        categories.append("dog")

In [5]:
df = pd.DataFrame({
    "filename": files,
    "category": categories
})

In [6]:
df.shape

(25000, 2)

In [7]:
df.head(5)

,filename,category
0,dog.3004.jpg,dog
1,cat.9175.jpg,cat
2,cat.7052.jpg,cat
3,dog.9368.jpg,dog
4,dog.3408.jpg,dog


In [8]:
from sklearn.model_selection import train_test_split
df_train, df_validation = train_test_split(df,test_size=0.2)

In [9]:
df_train.shape

(20000, 2)

In [10]:
df_validation.shape

(5000, 2)

In [11]:
df_train.head(5)

,filename,category
2445,cat.5058.jpg,cat
19374,dog.4537.jpg,dog
24180,dog.11934.jpg,dog
16940,cat.10902.jpg,cat
11700,cat.1202.jpg,cat


In [12]:
df_validation.head(5)

,filename,category
10375,dog.240.jpg,dog
8488,dog.5092.jpg,dog
16697,cat.1150.jpg,cat
21807,dog.11589.jpg,dog
10993,cat.10260.jpg,cat


In [13]:
total_train = df_train.shape[0]
total_validate = df_train.shape[0]
batch_size = 15

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(df_train,
                                                 "./dogs-vs-cats/train/",x_col="filename",y_col="category",
                                                 target_size=(Image_Height,Image_Width),
                                                 class_mode="categorical",
                                                 batch_size=batch_size)
validation_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
validation_generator = validation_datagen.flow_from_dataframe(df_validation,
                                                 "./dogs-vs-cats/train/",x_col="filename",y_col="category",
                                                 target_size=(Image_Height,Image_Width),
                                                 class_mode="categorical",
                                                 batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [15]:
model = Sequential()

model.add(Convolution2D(32,(3,3),input_shape=(Image_Height,Image_Width,Image_Channels),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3,3),input_shape=(Image_Height,Image_Width,Image_Channels),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3,3),input_shape=(Image_Height,Image_Width,Image_Channels),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

2022-08-01 16:15:57.277249: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-01 16:15:57.277674: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-01 16:15:57.279333: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0

In [17]:
model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

In [18]:
model.fit_generator(train_generator,validation_data=validation_generator, epochs=10)

/home/harshil/anaconda3/envs/python/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-08-01 16:15:57.631859: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-08-01 16:15:57.632301: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2303995000 Hz


Epoch 1/10
1334/1334 [==============================] - 711s 532ms/step - loss: 0.9677 - accuracy: 0.5866 - val_loss: 0.5893 - val_accuracy: 0.6982
Epoch 2/10
1334/1334 [==============================] - 674s 505ms/step - loss: 0.5641 - accuracy: 0.7198 - val_loss: 0.5867 - val_accuracy: 0.7418
Epoch 3/10
1334/1334 [==============================] - 843s 632ms/step - loss: 0.4981 - accuracy: 0.7621 - val_loss: 0.5802 - val_accuracy: 0.7024
Epoch 4/10
1334/1334 [==============================] - 591s 443ms/step - loss: 0.4770 - accuracy: 0.7745 - val_loss: 0.7746 - val_accuracy: 0.6602
Epoch 5/10
1334/1334 [==============================] - 520s 390ms/step - loss: 0.4398 - accuracy: 0.7979 - val_loss: 0.4760 - val_accuracy: 0.7840
Epoch 6/10
1334/1334 [==============================] - 560s 420ms/step - loss: 0.4216 - accuracy: 0.8064 - val_loss: 0.4474 - val_accuracy: 0.7932
Epoch 7/10
1334/1334 [==============================] - 548s 410ms/step - loss: 0.4130 - accuracy: 0.8160 - val_

In [19]:
model.save("model1_catsVSdogs_10epoch.h5")